# Group Project / Assignment 4: Instruction finetuning a Llama-3.2 model
**Assignment due 21 April 11:59pm**

Welcome to the fourth and final assignment for 50.055 Machine Learning Operations. The third and fourth assignment together form the course group project. You will continue the work on a chatbot which can answer questions about SUTD to prospective students.


**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**. The assignment is more open-ended than previous assignments, i.e. you have more freedom how to solve the problem and how to structure your code.
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment. If you work on another environment, minimally test your work on the SUTD Education Cluster.

**Rubric for assessment** 

Your submission will be graded using the following criteria. 
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. Creativity and innovation: in this assignment you have more freedom to design your solution, compared to the first assignments. You can show of your creativity and innovative mindset. 
6. There is a maximum of 310 points for this assignment.

**ChatGPT policy** 

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission Instructions are located at the bottom of the notebook.



### Finetuning LLMs

The goal of the assignment is to build a more advanced chatbot that can talk to prospective students and answer questions about SUTD.

We will finetune a smaller 1B LLM on question-answer pairs which we synthetically generate. Then we will compare the finetuned and non-finetuned LLMs with and without RAG to see if we were able to improve the SUTD chatbot answer quality. 

We'll be leveraging `langchain`, `llama 3.2` and `Google AI STudio with Gemini 2.0`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [Llama 3.2](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/)
- [Google AI Studio](https://aistudio.google.com/)

Note: Google AI Studio provides a lot of free tokens but has certain rate limits. Write your code in a way that it can handle these limits.

# Install dependencies
Use pip to install all required dependencies of this assignment in the cell below. Make sure to test this on the SUTD cluster as different environments have different software pre-installed.  

In [7]:
# QUESTION: Install and import all required packages
# The rest of your code should execute without any import or dependency errors.

# **--- ADD YOUR SOLUTION HERE (10 points) ---**

# Correct package installations
!pip install google-generativeai
!pip install transformers
!pip install langchain




# Generate training data
The first step of the assignment is generating synthetic question-answer pairs which can be used for finetuning an LLM model. 
Use the Google AI studio with the Gemini models to create -high-quality QA training data.


In [133]:
# QUESTION: Use langchain and the Google AI Studio APIs and a model from the Gemini 2.0 family
# to create a text-generation chain that can produce and parse JSON output.
# Test it by having the LLM generate a JSON array of 3 fruits

#--- ADD YOUR SOLUTION HERE (20 points)---
from typing import Optional, List, ClassVar
import google.generativeai as genai
from langchain.schema import LLMResult
from langchain.llms.base import LLM
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env


# Load API key
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Custom Gemini LLM wrapper
class GeminiLLM(LLM):
    model_name: ClassVar[str] = "gemini-2.0-flash"  # Latest fast model, not "gemini-2.0-flash"
    temperature: float = 0.7

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        model = genai.GenerativeModel(self.model_name)
        response = model.generate_content(prompt)
        return response.text

    @property
    def _llm_type(self) -> str:
        return "gemini-2.0-flash"

# LangChain setup
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define prompt with input variable
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}"
)

# Instantiate the chain
llm = GeminiLLM()
chain = LLMChain(llm=llm, prompt=prompt)

# Call with a variable




## Generate topics
When generating data, it is often helpful to guide the generation process through some hierachical structure. 
Before we create question-answer pairs, let's generate some topics which the questions should be about.



In [225]:
# QUESTION: Create a function 'generate_topics' which generates topics which prospective students might care about.
#
# Generate a list of 20 topics 

#--- ADD YOUR SOLUTION HERE (20 points)---

def generate_topics(num):
    # Run the chain with the question about generating a list of topics
    output = chain.run(question=f"Generate a list of {num} topics which prospective students might care about in list form, just the title of the topic padded with ,,, . Do not include any other text.")

    output = output.split(",,,")
    output = [topic.strip() for topic in output if topic.strip()]  # Remove empty strings and strip whitespace
    output = [topic for topic in output if topic]  # Remove empty strings
    output = [topic.replace(",,", "") for topic in output]  # Remove commas from topics
    return output




In [226]:
# test topic generation
print(generate_topics(num = 3))

['Academics', 'Campus Life', 'Career Opportunities']


In [228]:
import json

# Generate a list of 20 topics 
# We save a copy to disk and reload it from there if the file exists

output = generate_topics(num = 20)
print(output)

with open("topics.txt", "w") as f:

    for i in output:
        f.write(i+"\n") if i != "," else None




['Academic Programs', 'Faculty Expertise', 'Research Opportunities', 'Campus Life', 'Student Housing', 'Financial Aid & Scholarships', 'Tuition & Fees', 'Career Services', 'Internship Opportunities', 'Study Abroad Programs', 'Location & Surroundings', 'Student Clubs & Organizations', 'Diversity & Inclusion', 'Health & Wellness Resources', 'Safety & Security', 'Alumni Network', 'Technology & Facilities', 'Admission Requirements', 'Application Process', 'Campus Visit Options']


## Generate questions
Now generate a set of questions about each topic

In [215]:
# QUESTION: Create a function 'generate_questions' which generates quetions about a given topic. 
# Generate a list of 10 questions per topics. In total you should have 200 questions. 
#

def generate_questions(topic,num):
    # Run the chain with the question about generating a list of topics

            # Run the chain with the question about generating a list of questions
        output = chain.run(question=f"Generate a list of {num} questions about {topic} which prospective students might care about, don't print anything else, just questions padded with ,,, .", num=num)
        output = output.split(",,,")

        output_list = []
        
        for i in output[1:-1]:
            i = i.strip()
            if i != "":
                output_list.append(i)

        return output_list
            

       
 


#--- ADD YOUR SOLUTION HERE (20 points)---


In [216]:
# test it
print(generate_questions("Academic Reputation and Program Quality", 3))


["How does the program's curriculum reflect current industry trends and research?", "What is the faculty's research output and its impact on the field?", 'What are the career placement rates and average starting salaries for graduates of this program?']


In [241]:
# # QUESTION: Now let's put it together and generate 10 questions for each topic. Save the questions in a local file.

#--- ADD YOUR SOLUTION HERE (20 points)---
import csv
import time

with open("topics.txt", "r") as f_topics, open("questions.csv", "w", newline='') as f_csv:
    writer = csv.writer(f_csv)
    
    for topic in f_topics:
        topic = topic.strip()
        print(topic)
        print(question)
        questions = generate_questions(topic, 10)
        time.sleep(4)

        for question in questions:
            row = [topic, question]  
            writer.writerow(row)



Academic Programs
How does the university track and measure the success of its alumni network in supporting student outcomes?
Faculty Expertise
What are the program's learning outcomes and how are they assessed?
Research Opportunities
What are the faculty's areas of expertise and how do they align with my academic interests?
Campus Life
What are the potential career benefits of participating in undergraduate research (e.g., improved job prospects, graduate school admission)?
Student Housing
,What kind of recreational facilities are available on campus (e.g., gym, swimming pool, sports fields) and what intramural sports or fitness programs are offered?
Financial Aid & Scholarships
What is the process for moving in and out of student housing?
Tuition & Fees
Who can I contact if I have questions about my financial aid application or award?
Career Services
What resources are available to help me create a budget and manage my finances as a student?
Internship Opportunities
How early in my a

## Generate Answers

Now create answers for the questions. 

You can use the Google AI Studio Gemini model (assuming that they are good enough to generate good answers), your RAG system from assignment 3 or any other method you choose to generate answers for your question dataset.

Note: it is normal that some LLM calls fail, even with retry, so maybe you end up with less than 200 QA pairs but it should be at least 160 QA pairs.

In [255]:
# QUESTION: Generate answers to al your questions using Gemini, your SUTD RAG system or any other method.
# Split your dataset in to 80% training and 20% test dataset.
# Store all questions and answer pairs in a huggingface dataset `sutd_qa_dataset` and push it to your Huggingface hub. 

#--- ADD YOUR SOLUTION HERE (40 points)---

def generate_answer(question):
    # Run the chain with the question about generating a list of topics
    output = chain.run(question=f"Answer the following question in the context of SUTD (Singapore University of Technology and Design) in text, no **, no follow-up questions, just a straight response : {question}")
    return output






In [256]:
# test the chain
question = "When was SUTD founded?"

# Now run the answer generation chain
response = generate_answer(question)
print("\nModel Response:")
print(response)


Model Response:
SUTD was founded in 2009.



In [257]:
import csv
import time

with open("questions.csv", "r") as f_csv, open("answers.csv", "w", newline='') as f_answers:
    reader = csv.reader(f_csv)
    writer = csv.writer(f_answers)

    for row in reader:
        topic, question = row
        print(f"{topic}\n\"{question}\"")

        try:
            answer = generate_answer(question)
        except Exception as e:
            print(f"Error generating answer: {e}")
            answer = "Error generating answer"

        time.sleep(4)  # prevent rate limit
        writer.writerow([topic, question, answer])



Academic Programs
"What are the specific admission requirements for this program?"
Academic Programs
"What is the program's curriculum structure and what courses are required?"
Academic Programs
"Are there opportunities for internships, research, or study abroad within this program?"
Academic Programs
"What are the career prospects and typical job titles for graduates of this program?"
Academic Programs
"What is the average class size and what is the student-to-faculty ratio?"
Academic Programs
"What resources and support services are available to students in this program (e.g., tutoring, advising, career counseling)?"
Academic Programs
"Are there any opportunities for specialization or concentration within this program?"
Academic Programs
"How does this program incorporate practical experience or real-world application of the concepts taught?"
Academic Programs
"What is the program's accreditation status and how does that benefit students?"
Academic Programs
"What are the program's le

# Finetune Llama 3.2 1B model

Now use your SUTD QA dataset training data set to finetune a smaller Llama 3.2 1B LLM using parameter-efficient finetuning (PEFT). 
We recommend the unsloth library but you are free to choose other frameworks. You can decide the parameters for the finetuning. 
Push your finetuned model to Huggingface. 

Then we will compare the finetuned and non-finetuned LLMs with and without RAG to see if we were able to improve the SUTD chatbot answer quality. 


In [ ]:
# QUESTION: Finetune a Llama 3.2 1B model on the training split of your SUTD QA dataset.
# You need to prepare your dataset accordingly and set the hyperparameters for the training.
# Push your finetuned model to the Hugginface model hub {YOUR_HF_NAME}/llama-3.2-1B-sutdqa

#--- ADD YOUR SOLUTION HERE (50 points)---



In [ ]:
# QUESTION: Load a non-finetuned Llama 3.2 1B model and your finetuned SUTD QA Llama 3.2 1B model
# Ask it a simple test question (e.g. "What is special about SUTD?") to check that both models can generated answers

#--- ADD YOUR SOLUTION HERE (10 points)---



In [ ]:
# try out the llms

query = "What is special about SUTD?"

print("Question:", query)
response_base = llm_base.invoke(query,  pipeline_kwargs={"max_new_tokens": 512})
print("Answer base:", response_base)

print("---------")
response_finetune = llm_finetune.invoke(query, pipeline_kwargs={"max_new_tokens": 512})
print("Answer finetune:", response_finetune)


# Integrate and evaluate

Now integrate both the non-finetuned Llama 3.2 1B model and your finetuned model into your SUTD chatbot RAG system. 
Generate responses to the 20 questions you have collected in assignment 3 using these 4 appraoches
1. non-finetuned Llama 3.2 1B model without RAG
2. finetuned Llama 3.2 1B SUTD QA model without RAG
3. non-finetuned Llama 3.2 1B model with RAG
4. finetuned Llama 3.2 1B SUTD QA model with RAG

Compare the responses and decide what system produces the most accurate and high quality responses

In [ ]:
# QUESTION: Re-create the RAG chatbot system you have created in assignment 3 but with the Llama 3.2 1B (non-tuned and finetuned) models

#--- ADD YOUR SOLUTION HERE (40 points)---


# Bonus points: LLM-as-judge evaluation 

Implement an LLM-as-judge pipeline to assess the quality of the different system (finetuned vs. non-fintuned, RAG vs no RAG)



In [ ]:
# QUESTION: Implement an LLM-as-judge pipeline to assess the quality of the different system (finetuned vs. non-fintuned, RAG vs no RAG)

#--- ADD YOUR SOLUTION HERE (40 points)---

# Bonus points: chatbot UI

Implement a web UI frontend for your chatbot that you can demo in class. 


In [ ]:
# QUESTION: Implement a web UI frontend for your chatbot that you can demo in class. 

#--- ADD YOUR SOLUTION HERE (40 points)---

# End

This concludes assignment 4.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every group member should do the following submission steps:
1. Create a private github repository **sutd_5055mlop** under your github user.
2. Add your instructors as collaborator: ddahlmeier and lucainiaoge
3. Save your submission as assignment_04_GROUP_NAME.ipynb where GROUP_NAME is the name of the group you have registered. 
4. Push the submission files to your repo 
5. Submit the link to the repo via eDimensions



**Assignment due 21 April 2025 11:59pm**